<a href="https://colab.research.google.com/github/currencyfxjle/DeepLearning_PyTorch/blob/main/DeepLearning_PyTorch_JoseLizarraga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install torch torchvision

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
if torch.cuda.is_available():
    from torch.cuda.amp import GradScaler, autocast

# Transformaciones
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Cargar el dataset
dataset_path = '/content/drive/My Drive/Colab Notebooks/Tec/Deep Learning Utilizando PyTorch/Catsvsdogs_'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Dividir el dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

if torch.cuda.is_available():
    scaler = GradScaler()

def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        if torch.cuda.is_available():
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        running_loss += loss.item()

    train_acc = calculate_accuracy(train_loader, model)
    test_acc = calculate_accuracy(test_loader, model)
    print(f'Época {epoch+1}, Pérdida: {running_loss/len(train_loader)}, Entrenamiento Accuracy: {train_acc}%, Prueba Accuracy: {test_acc}%')


Época 1, Pérdida: 0.0056384520802071425, Entrenamiento Accuracy: 100.0%, Prueba Accuracy: 100.0%
Época 2, Pérdida: 0.0, Entrenamiento Accuracy: 100.0%, Prueba Accuracy: 100.0%
Época 3, Pérdida: 0.0, Entrenamiento Accuracy: 100.0%, Prueba Accuracy: 100.0%
Época 4, Pérdida: 0.0, Entrenamiento Accuracy: 100.0%, Prueba Accuracy: 100.0%
Época 5, Pérdida: 0.0, Entrenamiento Accuracy: 100.0%, Prueba Accuracy: 100.0%
